![図1.2 Sparkの紹介とインストール](images/PysparkInProduction.png)

# 本チャプターの目次

1. これまでと本レクチャーの違い
2. Pysparkを本番環境で動かす際の流れ
3. Spark Submitを動かす(Sparkをコマンドラインで実行する方法)
4. チューニングのコツ
   1. Spark Webインタフェースを用いたボトルネックの作成
   2. メモリへの登録

# Sparkを用いたバッチにおけるデータエンジニアリング一連の流れ(復習)

1. データソースの読み込み(今回は、人口統計データ(/dataset/jinkou.csv))　ETL(Extract Transform Load)で言うEの部分
2. 変換を行う(集計等を行う)　DataFrame処理 or SQL処理の２パターンで実行可能 ETL(Extract Transform Load)で言うTの部分
3. カラムナーフォーマットへ変換する ETL(Extract Transform Load)で言うTの部分
4. 出力したデータをみんなに見やすくするため(BIツールから参照できるように)テーブルを作成する ETL(Extract Transform Load)で言うLの部分

本チャプターではこの流れを、Spark-Submitを使った方法で実行してみようと思います。
ただし、テーブルは事前に作成しておくことにします。

# これまでと本レクチャーの違い
これまでは、ローカル環境での開発を想定していました。  
本レクチャーでは、本番環境で実行をするための準備を行っていきます。  

セクション3,4で行ってきた内容を、本番環境に適用することを考えていきます。

主に使うリソースは./pysaprk_batch/spark_etlフォルダ配下に入っています。  

# 各リソースの説明
- etl.sql(sparkSQL。データの変換を担当するSQL。セクション4で紹介した処理と同じだが、元号だけあえて可変引数にしている。spark_etl_sample.pyで読み込んで実行する)
- jinko_schema.json(jinko.csvのスキーマ定義が入っている。動的に行うためにはファイルで外出しすることが好ましい。spark_etl_sample.pyで読み込んで適用する)
- spark_etl_sample.py(データエンジニアリング一覧の流れが記載されている)

# PySparkを実際に本番環境で動かす際の流れ
ローカル端末で動かすことはわかったので、本チャプターはPySparkを実際に本番環境で実行するためのTipsを紹介していきます。  
メインのトピックとしては、

- スケジューラーで実行するために、sample_elt_sample.pyをコマンドベースでどのように動かすか？
- 本番で直面する問題や、チューニングする際に利用するツールやそのコツ

を紹介していきます。

# spark-submit でSparkを実行する

コマンドラインでSparkを実行するにはこの方法しかない。
構文としては

```
spark-submit xxxxx.py 引数 
```

で実行可能

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は次のチャプターで説明を行います。
spark = SparkSession.builder \
    .appName("chapter3") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
# テーブルは事前に作成しておきます(テーブル名はちょっとだけ変えてjinko_avg2としてます)
spark.sql(""" 
CREATE EXTERNAL TABLE2 IF NOT EXISTS default.jinko_avg2 ( male_avg double, female_avg double)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
LOCATION '/Users/yuki/pyspark_batch/dataset/parquet/';
""")
spark.sql(" show tables").show()

# spark_etl.pyを作成していきます
# ここからはノートブックではなくpythonエディター（ターミナル）へ移ります。

```
spark-submit ./spark_etl/spark_etl_sample.py -a hoge -s ./spark_etl/etl.sql -b GENGO=昭和 -t jinko -z ./spark_etl/jinko_schema.json -f /Users/yuki/pyspark_batch/dataset/jinko.csv
```

ターミナルを起動したらmetastore_dbが直下にあるところまで移動をしてください。
メタデータストアなので、ノートブックから作ったテーブル定義を利用することが可能だからです(今回だとjinko_avg2のテーブル定義を使います)。


実際にクラウド上の実行環境でspark-submitを行う場合は、比較的実行の設定値がラップされていることが多くあまり実行において気にすることはあまりない  
オンプレの場合は、細かな設定を行わなければならず苦労するポイントかも知れません。

# チューニングTips

Sparkというと、メモリに気をとられドライバーのメモリーやエグゼキューターのメモリについてのチューニング記事がたくさん出てきます。  
しかしながら、これらのメモリー設定がどうしても必要になるパターンはまれです(昔はよく設定していた)。

メモリー設定を弄りまわす前に確認したいいくつかのポイントを紹介します

1. Spark web インタフェースでのボトルネックの確認
2. メモリへの登録
3. repartition数を大きくしてみる
4. Executor(ノード)の台数を増やす(クラウドであれば容易)　メモリ設定などですごく迷うのであれば、さっとノード追加して動かしてしまったほうが結局安上がり



# Spark web インタフェースでのボトルネックの確認
Sparkのウェブインタフェースを眺めてみましょう

1. sparksessionを起動する
2. http://localhost:4040 へアクセスします

いくつかの処理を実行して、結果を見比べてみましょう。

In [ ]:
spark.sql(""" 

select * from jinko_avg2

""").show()

In [ ]:
spark.sql(""" 

select * from jinko_avg2 where kenmei='三重県'

""").show()

In [ ]:
spark.sql(""" 

select * from jinko_avg2 where male_avg > 3000000    

""").show()

# メモリへの登録
処理途中のデータはメモリへ登録することが可能です。  
何度も使い回す場合で比較的小さいデータはメモリ登録することで速度の向上が見込めます。

メモリの登録にはいくつか種類があります

1. dataframeをキャッシュする
2. テンポラリテーブルをキャッシュする



In [ ]:
# dataframeをキャッシュする場合
df=spark.sql(""" 

select * from jinko_avg2 where kenmei='三重県'

""")

# キャッシュする
df.cache()
# キャッシュ判定
df.is_cached

# 後続でdfを使うときはキャッシュから利用される

In [ ]:
# Spark利用の停止
spark.stop()
spark.sparkContext.stop()